In [23]:
import torch
import torchvision
import tqdm
import matplotlib.pyplot as plt

In [24]:
# 使用 GPU
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

In [25]:
# 数据预处理, 调整图片大小
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(),torchvision.transforms.Resize([224, 224])])

# 下载Fashion-MNIST训练集数据，并构建训练集数据载入器trainloader,每次从训练集中载入64张图片，每次载入都打乱顺序
trainset = torchvision.datasets.FashionMNIST('dataset/', download=False, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# 下载Fashion-MNIST测试集数据，并构建测试集数据载入器trainloader,每次从测试集中载入64张图片，每次载入都打乱顺序
testset = torchvision.datasets.FashionMNIST('dataset/', download=False, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

In [26]:
def make_net(cfg):
    layers = []
    pre = 1
    for cur in cfg:
        if cur == "M":
            layers.append(torch.nn.MaxPool2d(kernel_size=2, stride=2)) # 加入一个最大汇聚成
        else:
            layers.append(torch.nn.Conv2d(pre, cur, kernel_size=3, padding=1)) # 加入一个卷积层
            layers.append(torch.nn.ReLU()) # 激活函数
            pre = cur
    return torch.nn.Sequential(*layers)

In [27]:
class VGG(torch.nn.Module):
    def __init__(self, conv_arch):
        super(VGG, self).__init__()
        self.chunks = make_net(conv_arch)
        self.output = torch.nn.Sequential( torch.nn.Flatten(),
            torch.nn.Linear(512 * 7 * 7, 4096), torch.nn.ReLU(), torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 4096), torch.nn.ReLU(), torch.nn.Dropout(0.5),
            torch.nn.Linear(4096, 10)
        )
        
    def forward(self, x):
        x = self.chunks(x)
        x = self.output(x)
        return x

In [28]:
cfg = {
    "vgg11": [64, "M", 128, "M", 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]
}

In [29]:
net = VGG(cfg["vgg11"])
net = net.to(device=try_gpu())

net.train()

cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

loss_data = {
    "epoch" : [],
    "loss" : []
}

i = 0
for epoch in tqdm.tqdm(range(10)):
    loss_sum = 0
    for images, labels in trainloader:
        images = images.to(device=try_gpu())
        labels = labels.to(device=try_gpu())
        y_predict = net(images)
        optimizer.zero_grad()
        loss = cost( y_predict, labels )
        loss.backward()
        optimizer.step()
        i = i + 1
        loss_sum = loss_sum + float(loss)
    loss_data["epoch"].append(epoch)
    loss_data["loss"].append(loss_sum / i)
    i = i + 1

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
print(loss_data)
plt.plot(loss_data["epoch"], loss_data["loss"])
plt.show()

In [ ]:
net.eval()
yhat = []
epoch = 0
accuracy_sum = 0
for images, labels in testloader:
    images = images.to(device=try_gpu())
    labels = labels.to(device=try_gpu())
    y_predict = net(images)
    y_predict = torch.argmax(y_predict, dim = 1)
    accuracy = torch.eq(labels, y_predict).float().mean()
    accuracy_sum = accuracy_sum + accuracy
    epoch = epoch + 1
print("平均精准度：%f" % (accuracy_sum / epoch))